# [Advent of Code 2021: Day 5](https://adventofcode.com/2021/day/5)
[puzzle input](https://adventofcode.com/2021/day/5/input)

## \-\-\- Day 5: Hydrothermal Venture \-\-\-

You come across a field of [hydrothermal vents](https://en.wikipedia.org/wiki/Hydrothermal_vent) on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in **lines**; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2

```

Each line of vents is given as a line segment in the format `x1,y1 -> x2,y2` where `x1`,`y1` are the coordinates of one end the line segment and `x2`,`y2` are the coordinates of the other end. These line segments include the points at both ends. In other words:

*   An entry like `1,1 -> 1,3` covers points `1,1`, `1,2`, and `1,3`.
*   An entry like `9,7 -> 7,7` covers points `9,7`, `8,7`, and `7,7`.

For now, **only consider horizontal and vertical lines**: lines where either `x1 = x2` or `y1 = y2`.

So, the horizontal and vertical lines from the above list would produce the following diagram:

```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....

```

In this diagram, the top left corner is `0,0` and the bottom right corner is `9,9`. Each position is shown as **the number of lines which cover that point** or `.` if no line covers that point. The top\-left pair of `1`s, for example, comes from `2,2 -> 2,1`; the very bottom row is formed by the overlapping lines `0,9 -> 5,9` and `0,9 -> 2,9`.

To avoid the most dangerous areas, you need to determine **the number of points where at least two lines overlap**. In the above example, this is anywhere in the diagram with a `2` or larger \- a total of **`5`** points.

Consider only horizontal and vertical lines. **At how many points do at least two lines overlap?**

In [1]:
import unittest
from fractions import Fraction
from IPython.display import Markdown, display

from aoc_puzzle import AocPuzzle

class Puzzle(AocPuzzle):
    
    def parse_line(self, raw_data):
        coord_str_set = raw_data.split(' -> ')
        coord_set = []
        for coord_str in coord_str_set:
            coord = tuple(map(int, coord_str.split(',')))
            coord_set.append(coord)
        return coord_set
    
    def parse_data(self, raw_data):
        self.data = list(map(self.parse_line, raw_data.split('\n')))
        
    def get_slope(self, coord1, coord2):
        x1, y1 = coord1
        x2, y2 = coord2
        return((x1 - x2, y1 - y2))
    
    def incriment_coord(self, coord, step):
        x,y = coord
        xstep, ystep = step
        return (int(x+xstep), int(y+ystep))
    
    def decriment_coord(self, coord, step):
        x,y = coord
        xstep, ystep = step
        return (x-xstep, y-ystep)
    
    def get_step(self, coord):
        x, y = coord
        
        if x is 0:
            xcd = 0
            ysign = y / abs(y)
            ycd = ysign
        elif y is 0:
            xsign = x / abs(x)
            xcd = xsign
            ycd = 0
        else:
            xsign = x / abs(x)
            ysign = y / abs(y)
            cf = Fraction(x, y)
            xcd = abs(cf.numerator) * xsign
            ycd = cf.denominator * ysign
            
        return (xcd, ycd)
        
    def get_step_to(self, coord1, coord2):
        """ Returns a step for coordinate 1 that is in the direction of coordinate 2 """
        if coord1 is coord2:
            return (0,0)
        
        x1, y1 = coord1
        x2, y2 = coord2        
        dist = self.get_slope(coord2, coord1)
        xstep, ystep = self.get_step(dist)
        
        return (xstep, ystep)
        
    def expand_coord_set(self, coord_set):
        coord1, coord2 = coord_set
        step = self.get_step_to(coord1, coord2)
        coords_list = [coord1]
        next_coord = self.incriment_coord(coord1, step)
        while next_coord != coord2:
            coords_list.append(next_coord)
            next_coord = self.incriment_coord(next_coord, step)
        coords_list.append(coord2)
        
        return coords_list
    
    def is_hort_vert_set(self, coord_set):
        coord1, coord2 = coord_set
        x1, y1 = coord1
        x2, y2 = coord2
        
        if x1 == x2 or y1 == y2:
            return True
        else:
            return False
        
    def get_all_coords(self, horz_vert_only):
        coords_list = []
        for coord_set in self.data:
            if horz_vert_only:
                if self.is_hort_vert_set(coord_set):
                    set_coords = self.expand_coord_set(coord_set)
                    coords_list += set_coords
            else:
                set_coords = self.expand_coord_set(coord_set)
                coords_list += set_coords
                
        return coords_list
    
    def count_dupplicates(self, coords_list):
        coord_counts = {}
        
        for coord in coords_list:
            coord_counts.setdefault(coord, 0)
            coord_counts[coord] += 1
            
        total = 0
        for coord, count in coord_counts.items():
            if count > 1:
                total += 1
        return total            
            
    def run(self, horz_vert_only=True, output=False):
        coords_list = self.get_all_coords(horz_vert_only)
        result = self.count_dupplicates(coords_list)
        
        if output:
            display(Markdown(f'### Result is `{result}`'))            
        return result
        

class TestBasic(unittest.TestCase):

    def test_parse_data(self):
        in_data = '0,9 -> 5,9\n8,0 -> 0,8\n9,4 -> 3,4\n2,2 -> 2,1\n7,0 -> 7,4\n6,4 -> 2,0\n0,9 -> 2,9\n3,4 -> 1,4\n0,0 -> 8,8\n5,5 -> 8,2'
        exp_out = [[(0,9),(5,9)],[(8,0),(0,8)],[(9,4),(3,4)],[(2,2),(2,1)],[(7,0),(7,4)],[(6,4),(2,0)],[(0,9),(2,9)],[(3,4),(1,4)],[(0,0),(8,8)],[(5,5),(8,2)]]
        puzzle = Puzzle(in_data)
        self.assertEqual(puzzle.data, exp_out)
        
    def test_puzzle(self):
        input_data = ['0,9 -> 5,9\n8,0 -> 0,8\n9,4 -> 3,4\n2,2 -> 2,1\n7,0 -> 7,4\n6,4 -> 2,0\n0,9 -> 2,9\n3,4 -> 1,4\n0,0 -> 8,8\n5,5 -> 8,2']
        exp_output = [5]
        for in_data, exp_out in tuple(zip(input_data, exp_output)):
            puzzle = Puzzle(in_data)
            self.assertEqual(puzzle.run(), exp_out)
        
unittest.main(argv=[""], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


In [2]:
puzzle = Puzzle("input/d05.txt")
puzzle.run(output=True)

### Result is `7468`

7468

## \-\-\- Part Two \-\-\-

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider **diagonal lines**.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

*   An entry like `1,1 -> 3,3` covers points `1,1`, `2,2`, and `3,3`.
*   An entry like `9,7 -> 7,9` covers points `9,7`, `8,8`, and `7,9`.

Considering all lines from the above example would now produce the following diagram:

```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....

```

You still need to determine **the number of points where at least two lines overlap**. In the above example, this is still anywhere in the diagram with a `2` or larger \- now a total of **`12`** points.

Consider all of the lines. **At how many points do at least two lines overlap?**

In [3]:
class TestBasic(unittest.TestCase):
        
    def test_puzzle(self):
        input_data = ['0,9 -> 5,9\n8,0 -> 0,8\n9,4 -> 3,4\n2,2 -> 2,1\n7,0 -> 7,4\n6,4 -> 2,0\n0,9 -> 2,9\n3,4 -> 1,4\n0,0 -> 8,8\n5,5 -> 8,2']
        exp_output = [12]
        for in_data, exp_out in tuple(zip(input_data, exp_output)):
            puzzle = Puzzle(in_data)
            self.assertEqual(puzzle.run(horz_vert_only=False), exp_out)
        
unittest.main(argv=[""], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [4]:
puzzle = Puzzle("input/d05.txt")
puzzle.run(output=True, horz_vert_only=False)

### Result is `22364`

22364